In [1]:
%load_ext autoreload
%autoreload 2

import os, sys, shutil, bz2, copy
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np

### Plotting imports ###
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as mcolors
import matplotlib as mpl
from matplotlib.patches import StepPatch
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import logomaker as lm
palette = list(mcolors.TABLEAU_COLORS.keys())
sns.set_theme(style="ticks", palette="muted")
sns.set_context("talk")
%matplotlib inline

/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# Navigate back to NBdir in case of re-running a code block:
if not 'NBdir' in globals():
    NBdir = os.getcwd()
print('Notebook is in: {}'.format(NBdir))
os.chdir(NBdir)  # If you changed the current working dir, this will take you back to the notebook dir.

# Define the path to the repo folder.
# Change if necessary.
homedir = '/'.join(NBdir.split('/')[0:-2])
print('Repo is in: {}'.format(homedir))
sys.path.insert(1, homedir)
from src.misc import index_to_sample_df, downsample_raw_input, read_tRNAdb_info, sample_df_to_dict
from src.read_processing import AR_merge, BC_split, Kmer_analysis, BC_analysis, UMI_trim
from src.alignment import SWIPE_align
from src.stats_collection import STATS_collection
from src.plotting import TRNA_plot
from src.transcript_mutations import TM_analysis

# These are default folder names for data and raw fastq files
# relative to the folder in which this notebook is in:
data_dir = 'data'
seq_dir = 'raw_fastq'
seq_dir_noDS = seq_dir # Not downsampled

# These folder names are used in subsequent processing steps
# to dump data. Best to not change:
AdapterRemoval_dir = 'AdapterRemoval'
BC_dir = 'BC_split'
UMI_dir = 'UMI_trimmed'
align_dir = 'SWalign'
stats_dir = 'stats_collection'
TM_dir = 'transcript_mutations'
plotting_dir = 'plotting'
tRNA_database = dict()
tRNA_database['human'] = '{}/tRNA_database/human/hg38-tRNAs.fa'.format(homedir)
tRNA_database['mouse'] = '{}/tRNA_database/mouse/mm10-tRNAs.fa'.format(homedir)
tRNA_database['ecoli'] = '{}/tRNA_database/ecoli/ecoli-tRNAs.fa'.format(homedir)
tRNA_database_masked = dict()
tRNA_database_masked['human'] = '{}/tRNA_database_masked/human/human-tRNAs.fa'.format(homedir)
tRNA_database_masked['ecoli'] = '{}/tRNA_database_masked/ecoli/ecoli-tRNAs.fa'.format(homedir)
# Read information (length, codon etc) of tRNAs into dictionary:
tRNA_data = read_tRNAdb_info(tRNA_database)
SWIPE_score_mat = '{}/utils/nuc_score-matrix.txt'.format(homedir)
SWIPE_score_mat2 = '{}/utils/nuc_score-matrix_2.txt'.format(homedir) # For masked reference sequences
# tRNA sequencing yields many duplicated reads.
# Adding these commonly seen sequences to a list prevents duplicated alignment:
common_seqs = '{}/utils/common-seqs.fasta.bz2'.format(homedir)

# Define minimum read length based on minimum insert size:
MIN_INSERT_LEN = 10
UMI_LEN = 10
BC_MAX_LEN = 15
MIN_READ_LEN = MIN_INSERT_LEN + UMI_LEN + BC_MAX_LEN
print('Using minimum read length: {} (after merge)'.format(MIN_READ_LEN))

# Read index information:
# index_list_fnam = 'index_list.xlsx'
index_list_fnam = 'index_list_with-old.xlsx'

index_df = pd.read_excel('{}/utils/{}'.format(homedir, index_list_fnam))

Notebook is in: /home/sulab/tRNA-charge-seq/projects/tRNAseq_mtests2
Repo is in: /home/sulab/tRNA-charge-seq
Using minimum read length: 35 (after merge)


In [3]:
### Input settings ###
sample_list_fnam = 'sample_list.xlsx'
sample_df = pd.read_excel('{}/{}'.format(NBdir, sample_list_fnam))
# Add barcode sequences:
sample_df = index_to_sample_df(sample_df, index_df)
# Read elementary info (replicate, barcode, species)
# for each unique sample name into a dictionary:
sample_dict = sample_df_to_dict(sample_df)
# Get filenames from the sample information:
inp_file_df = sample_df[['fastq_mate1_filename', 'fastq_mate2_filename', 'P5_index', 'P7_index', 'P5_index_seq', 'P7_index_seq']].copy().drop_duplicates().reset_index(drop=True)

# Downsample:
if False:
    sample_df, inp_file_df, seq_dir = downsample_raw_input(sample_df, inp_file_df, NBdir, data_dir, seq_dir_noDS, downsample_absolute=1e4)

# Make a dictionary with paths used for data processing:
dir_dict = dict(NBdir = NBdir,
                data_dir = data_dir,
                seq_dir = seq_dir,
                AdapterRemoval_dir = AdapterRemoval_dir,
                BC_dir = BC_dir,
                UMI_dir = UMI_dir,
                align_dir = align_dir,
                stats_dir = stats_dir,
                TM_dir = TM_dir,
                plotting_dir = plotting_dir)

In [4]:
### Run AdapterRemoval ###
AR_obj = AR_merge(dir_dict, inp_file_df, MIN_READ_LEN, overwrite_dir=False, \
                  check_input=False, AR_threads=6)
inp_file_df = AR_obj.run_parallel(n_jobs=3, overwrite=False)

Using existing folder because overwrite set to false: /home/sulab/tRNA-charge-seq/projects/tRNAseq_mtests2/data/AdapterRemoval


In [5]:
### Split files based on barcodes ###
BCsplit_obj = BC_split(dir_dict, sample_df, inp_file_df, overwrite_dir=False)
sample_df, inp_file_df = BCsplit_obj.run_parallel(n_jobs=12, load_previous=True)

Using existing folder because overwrite set to false: /home/sulab/tRNA-charge-seq/projects/tRNAseq_mtests2/data/BC_split
Loaded results from previous run... Not running barcode split.


In [6]:
### Generate UMI stats and write final trimmed tRNA sequences ###
UMItrim_obj = UMI_trim(dir_dict, sample_df, overwrite_dir=True)
sample_df = UMItrim_obj.run_parallel(n_jobs=12, load_previous=False)
sample_df.head(3)

Downsampling UMI trimmed sequences to maximum 2000000 reads.


,sample_name_unique,sample_name,replicate,fastq_mate1_filename,fastq_mate2_filename,P5_index,P7_index,barcode,species,plot_group,hue_name,hue_value,hue_order,P5_index_seq,P7_index_seq,barcode_seq,N_total,N_CC,N_CCA,N_CCA+CC,CCA+CC_percent_total,percent_CCA,N_after_trim,N_UMI_observed,N_UMI_expected,percent_seqs_after_UMI_trim,percent_UMI_obs-vs-exp,N_after_downsample
0,U1,U1,1,2023-06-06/P03_R1.fastq.bz2,2023-06-06/P03_R2.fastq.bz2,D501,D703,l1N,human,CC_lig,Barcode,l1N,1,AGGCTATA,CGCTCATT,TNNNNNNGACTA,7589045,1371457,6481,1377938,18.156935,0.470340,7410594,518800,524287.618942,97.648571,98.953319,2000000
1,U6,U6,1,2023-06-06/P03_R1.fastq.bz2,2023-06-06/P03_R2.fastq.bz2,D501,D703,l6Sp,human,CC_lig,Barcode,l6Sp,2,AGGCTATA,CGCTCATT,GGCTGCCATGCTACAG,506580,442970,51232,494202,97.556556,10.366611,496014,204548,320726.407557,97.914248,63.776476,496014
2,T100_1,T100,1,2023-06-06/P13_R1.fastq.bz2,2023-06-06/P13_R2.fastq.bz2,D504,D702,l1Sp,ecoli,ecoli_mix_ctr,Percentage,100,1,TCAGAGCC,TCCGGAGA,GGCTGCCATGCGACTA,4759577,291879,4417087,4708966,98.936649,93.801633,4664217,517392,524216.227598,97.996461,98.698204,2000000


In [7]:
### Align reads to database of reference tRNAs ###
align_obj = SWIPE_align(dir_dict, tRNA_database_masked, sample_df, SWIPE_score_mat2, \
                        gap_penalty=6, extension_penalty=3, min_score_align=15, \
                        common_seqs=None, overwrite_dir=True)
sample_df = align_obj.run_parallel(n_jobs=12, overwrite=True, load_previous=False)
sample_df.head(3)

Running Swipe on:  U6  T0  U1  T88  T100_1  T13  K05m  T50  T75  L01  T100_2  T25  K2m  K32m  K4m  K8m  K64m  K16m  L02  K1m

,sample_name_unique,sample_name,replicate,fastq_mate1_filename,fastq_mate2_filename,P5_index,P7_index,barcode,species,plot_group,hue_name,hue_value,hue_order,P5_index_seq,P7_index_seq,barcode_seq,N_total,N_CC,N_CCA,N_CCA+CC,CCA+CC_percent_total,percent_CCA,N_after_trim,N_UMI_observed,N_UMI_expected,percent_seqs_after_UMI_trim,percent_UMI_obs-vs-exp,N_after_downsample,N_mapped,percent_single_annotation,percent_multiple_annotation,percent_multiple_codons,Mapping_percent
0,U1,U1,1,2023-06-06/P03_R1.fastq.bz2,2023-06-06/P03_R2.fastq.bz2,D501,D703,l1N,human,CC_lig,Barcode,l1N,1,AGGCTATA,CGCTCATT,TNNNNNNGACTA,7589045,1371457,6481,1377938,18.156935,0.470340,7410594,518800,524287.618942,97.648571,98.953319,2000000,1787738,62.973713,37.026287,6.138819,89.38690
1,U6,U6,1,2023-06-06/P03_R1.fastq.bz2,2023-06-06/P03_R2.fastq.bz2,D501,D703,l6Sp,human,CC_lig,Barcode,l6Sp,2,AGGCTATA,CGCTCATT,GGCTGCCATGCTACAG,506580,442970,51232,494202,97.556556,10.366611,496014,204548,320726.407557,97.914248,63.776476,496014,436339,61.380257,38.619743,5.142332,87.96909
2,T100_1,T100,1,2023-06-06/P13_R1.fastq.bz2,2023-06-06/P13_R2.fastq.bz2,D504,D702,l1Sp,ecoli,ecoli_mix_ctr,Percentage,100,1,TCAGAGCC,TCCGGAGA,GGCTGCCATGCGACTA,4759577,291879,4417087,4708966,98.936649,93.801633,4664217,517392,524216.227598,97.996461,98.698204,2000000,1818546,98.077200,1.922800,0.025680,90.92730


In [8]:
### Collect alignment statistics ###
stats_obj = STATS_collection(dir_dict, tRNA_data, sample_df, common_seqs=None, \
                             overwrite_dir=True)
stats_df = stats_obj.run_parallel(n_jobs=12, load_previous=False)
stats_df.head(3)

,sample_name_unique,sample_name,replicate,barcode,species,tRNA_annotation,tRNA_annotation_len,unique_annotation,5p_cover,align_3p_nt,codon,anticodon,amino_acid,align_gap,fmax_score>0.9,count,UMIcount,UMI_percent_exp
0,U1,U1,1,l1N,human,Escherichia_coli_str_K_12_substr_MG1655_tRNA-e...,76,True,False,A,AAA,TTT,eColiLys,False,True,1,1,100.000000
1,U1,U1,1,l1N,human,Homo_sapiens_mito_tRNA-Ala-TGC,72,True,False,C,GCA,TGC,Ala,False,False,7,7,100.000572
2,U1,U1,1,l1N,human,Homo_sapiens_mito_tRNA-Ala-TGC,72,True,False,C,GCA,TGC,Ala,False,True,54,54,100.005055


In [9]:
### Generate standard tRNAseq data plots ###
plot_obj = TRNA_plot(dir_dict, sample_df, overwrite_dir=True, pull_default=False)

In [10]:
# Ecoli control:
plot_obj.plot_Ecoli_ctr(min_obs=100)

/home/sulab/tRNA-charge-seq/src/plotting.py:301: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()


In [11]:
# Codon abundance barchart:
plot_obj.plot_abundance(plot_type='codon', plot_name='codon_abundance_grp', \
                        group=True, min_obs=500)

# Codon charge barchart:
plot_obj.plot_abundance(plot_type='codon', plot_name='codon_charge_grp', \
                        group=True, min_obs=500, charge_plot=True)


Now plotting sample/group:  AARS_kinetics  CC_lig  ecoli_mix_ctr  lig_test
Now plotting sample/group:  AARS_kinetics  CC_lig  ecoli_mix_ctr  lig_test

In [12]:
# Plot of abundance correlation:
plot_obj.plot_abundance_corr(sample_unique_pairs=[['L01', 'K64m', 'T88', 'T13', \
                                                   'U1',  'U6',   'U6',  'U1',  'U1'], \
                                                  ['L02', 'K1m',  'T0',  'T0', \
                                                   'U6',  'L01',  'L02', 'L01', 'L02']], \
                             plot_type='codon', plot_name='codon_abundance_corr', \
                             min_obs=500, charge_plot=False)


Now plotting sample pairs:  (L01 - L02)  (K64m - K1m)  (T88 - T0)  (T13 - T0)  (U1 - U6)  (U6 - L01)  (U6 - L02)  (U1 - L01)  (U1 - L02)

In [13]:
# Coverage plots for cyto/mito transcripts:
plot_obj.plot_coverage(compartment='cyto', plot_type='behrens', y_norm=True, \
                       plot_name='cov_plot_cyto_behrens_norm', n_jobs=8)

plot_obj.plot_coverage(compartment='mito', plot_type='behrens', y_norm=True, \
                       plot_name='cov_plot_mito_behrens_norm', n_jobs=8)


Now collecting data for sample:  U1  U6  T100_1  T88  T75  T50  T25  T13  T100_2

KeyError: 'Ile2'

In [ ]:
# UMI logo:
plot_obj.plot_UMI_logo(n_jobs=4)

In [ ]:
# Non-template nucleotides:
plot_obj.plot_non_temp(end='5p', plot_name='_5p-non-template_logo', \
                       seq_len_percentile=99, n_jobs=8)

plot_obj.plot_non_temp(end='3p', plot_name='_3p-non-template_logo', \
                       seq_len_percentile=99.9, n_jobs=8)

plot_obj.plot_non_temp(end='3p', plot_name='_3p-non-template_3p-cover_logo', \
                       seq_len_percentile=99.9, _3p_cover=True, n_jobs=8)

In [ ]:
### Perform transcript mutation analysis ###
TM_obj = TM_analysis(dir_dict, sample_df, tRNA_database, pull_default=False, \
                     common_seqs=None, ignore_common_count=False, \
                     overwrite_dir=True)

In [ ]:
# Find mutations and save them:
if True:
    TM_obj.find_muts(n_jobs=8, unique_anno=True)
    TM_obj.pickle_muts_write(pickle_name='saved_muts_unique-anno.pickle')
else:
    TM_obj.pickle_muts_read(pickle_name='saved_muts_unique-anno.pickle')